# Работа со сложными данными и обучение без учителя

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import (
                                    LabelEncoder, OneHotEncoder, label_binarize, OrdinalEncoder, 
                                    StandardScaler, QuantileTransformer, PowerTransformer, MinMaxScaler, RobustScaler
                                  )

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve, classification_report, recall_score, precision_score,
                             log_loss, brier_score_loss)

from category_encoders import TargetEncoder, WOEEncoder, HashingEncoder

In [ ]:
df = pd.read_csv('Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
RANDOM_STATE = 177013

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Exited'], axis=1),
                                                    df['Exited'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    stratify=df['Exited']
                                                    )

## Кодирование категорий

### One-Hot encoding

Простой и универсальный метод: каждое возможное значение можно описать бинарным признаком. Для N значений достаточно N-1 таких признаков.

Для всего датасета через pandas:

In [ ]:
pd.get_dummies(X_train['Geography'], drop_first=True)

Если у вас отдельные сеты, метод из pandas ненадежен, поскольку не гарантируется, что везде присутствуют все возможные значения.

Через sklearn:

In [ ]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
encoder.fit_transform(X_train[['Geography']])

### Label encoding

Если порядок категорий имеет смысл, можно закодировать их числами по возрастанию. Деревянные модели неплохо понимают такое кодирование, а вот для прочих использовать его не стоит.

Кодирование в алфавитном порядке:

In [ ]:
encoder = OrdinalEncoder()
X_train['geo_encoded'] = encoder.fit_transform(X_train[['Geography']])
X_train[['Geography', 'geo_encoded']].head()

Кодирование в порядке появления:

In [ ]:
X_train['geo_encoded'], uniques = pd.factorize(X_train['Geography'])
X_train[['Geography', 'geo_encoded']].head()

В произвольном порядке:

In [ ]:
X_train['geo_encoded'] = label_binarize(X_train['Geography'], classes=['Germany', 'France', 'Spain']).argmax(axis=1)
X_train[['Geography', 'geo_encoded']].head()

### Frequency encoding

In [ ]:
X_train['geo_encoded'] = st.rankdata(X_train['Geography'], method='average')
X_train[['Geography', 'geo_encoded']].head()

### Weight of evidence

Предназначен для задачи бинарной классификации. По сути является логарифмом отношения долей положительного и отрицательного класса для данного значения категории.

In [ ]:
encoder = WOEEncoder()
X_train['geo_encoded'] = encoder.fit_transform(X_train['Geography'], y_train)
X_train[['Geography', 'geo_encoded']].head(10)

**Этот метод использует целевой признак! Фитить его следует только на обучающей выборке после разбиения!**

### Mean-target encoding

Заменяет категорию на среднее значение целевого признака по этой категории. Хорошо подходит для случаев большой кардинальности, однако следует быть осторожным в плане утечки целевого признака для очень редких категорий. В `category_encoders` есть более робастные реализации, например, `LeaveOneOutEncoder()`.

In [ ]:
encoder = TargetEncoder()
X_train['geo_encoded'] = encoder.fit_transform(X_train['Geography'], y_train)
X_train[['Geography', 'geo_encoded']].head(10)

**Этот метод использует целевой признак! Фитить его следует только на обучающей выборке после разбиения!**

### Хеширование

Этот метод подходит, если категорий много, но некоторая потеря информации допустима.

In [ ]:
encoder = HashingEncoder()
encoder.fit_transform(X_train['Geography']).head(10)

Более подробную блок-схему выбора метода для кодировки можно найти на https://innovation.alteryx.com/encode-smarter/

### Кодирование внутри конвейера

In [ ]:
cat_columns = ['Geography', 'Gender']
useless_columns = ['CustomerId', 'RowNumber', 'Surname', 'geo_encoded']

In [ ]:
# Предобработка по группам признаков:
transformers = [
                    ("encoder", OneHotEncoder(drop='first'), cat_columns),
                    ("drop", "drop", useless_columns),
               ]

preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough', n_jobs=-1)

In [ ]:
def optimize(model, params, X, y):
    name = f'{type(model).__name__}'
    print(f'Оптимизация {name}...')
    pipe = Pipeline([
                        ('preprocessor', preprocessor),
                        ('scaler', None),                
                        ('model', model)
                    ])
    gcv = GridSearchCV(pipe, params, cv=4, scoring='neg_log_loss', n_jobs=-1)
    gcv.fit(X, y)
    print(f'Лучшие гиперпараметры: {dict(gcv.best_params_)}')
    print(f'Скор: {(-gcv.best_score_):.2f}')
    return gcv.best_estimator_

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
boost_params = {
                    'model__max_depth' : range(1, 10),
                    'model__max_iter' : [200, 500, 1000],
                }

In [ ]:
best_hgb = optimize(HistGradientBoostingClassifier(random_state=RANDOM_STATE), boost_params, X_train, y_train)

In [ ]:
def calculate_metrics(target_test, probabilities):
    cmatrix = confusion_matrix(target_test, probabilities > 0.5)

    ap = average_precision_score(target_test, probabilities)
    fpr, tpr, _ = roc_curve(target_test, probabilities)
    roc_auc = roc_auc_score(target_test, probabilities)

    precision, recall, thresholds = precision_recall_curve(target_test, probabilities)
    f_scores = 2 * recall * precision / (recall + precision)
    f_scores = np.nan_to_num(f_scores)
    best_thresh = thresholds[np.argmax(f_scores)]
    best_f = np.max(f_scores)
    best_acc = accuracy_score(target_test, (probabilities > best_thresh))
    best_cmatrix = confusion_matrix(target_test, (probabilities > best_thresh))

    return best_f, roc_auc, best_acc, ap, best_thresh, fpr, tpr, recall, precision, cmatrix, best_cmatrix

In [ ]:
def visualize(target_test, probabilities):
    fig, axes = plt.subplots(1, 2, figsize=(12,5))
    axes[0].plot([0, 1], linestyle='--')
    axes[1].plot([0.5, 0.5], linestyle='--')

    best_f, roc_auc, acc, ap, best_thresh, fpr, tpr, recall, precision, cmatrix, best_cmatrix = calculate_metrics(target_test, probabilities)
    print(f'ROC_AUC: {roc_auc:.2f}, AP (PR_AUC): {ap:.2f}, наилучший F1: {best_f:.2f} с порогом {best_thresh:.2f} (accuracy {acc:.2f})')
    axes[0].plot (fpr, tpr);
    axes[1].plot (recall, precision);

    axes[0].set (xlabel='FPR', ylabel='TPR', title='ROC-кривая', xlim=(0,1), ylim=(0,1))
    axes[1].set (xlabel='Recall', ylabel='Precision', title='PR-кривая', xlim=(0,1), ylim=(0,1))
    plt.show()
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    sns.heatmap(cmatrix, ax=axes[0], annot=True, cmap='Blues', fmt='d').set(title='Матрица ошибок', xlabel='Предсказание', ylabel='Реальность')
    sns.heatmap(best_cmatrix, ax=axes[1], annot=True, cmap='Blues', fmt='d').set(title='Матрица ошибок (оптимальный порог)', xlabel='Предсказание', ylabel='Реальность')

    
    return best_thresh

In [ ]:
visualize(y_test, best_hgb.predict_proba(X_test)[:,1])

In [ ]:
encoder_list = [
                OneHotEncoder(drop='first'),
                OrdinalEncoder(),
                WOEEncoder(),
                TargetEncoder(),
               ]

In [ ]:
boost_params = {
                    'preprocessor__encoder' : encoder_list, 
                    'model__max_depth' : range(1, 10),
                    'model__max_iter' : [200, 500, 1000],
                }

In [ ]:
best_hgb = optimize(HistGradientBoostingClassifier(random_state=RANDOM_STATE), boost_params, X_train, y_train)

## Простое кодирование текстов

Воспользуемся датасетом твитов о стихийных бедствиях и катастрофах:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

In [ ]:
df = pd.read_csv('disaster_tweets.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['location'].value_counts()

In [ ]:
text_columns = 'text'
useless_columns = ['id', 'keyword', 'location']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'], axis=1),
                                                    df['target'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    stratify=df['target']
                                                    )

### Bag of words

Самый простой способ - найти все уникальные слова и выделить бинарный признак на наличие каждого в тексте.

Недостатки достаточно очевидные:

- совершенно не учитывается порядок слов;
- уникальных слов может быть много - придется урезать число новых признаков в пользу самых частых.

Тем не менее, во многих случаях этот способ неплохо работает! В sklearn он представлен `sklearn.feature_extraction.text.CountVectorizer()`. При желании можно даже провести обратное преобразование и найти ключевые слова с помощью feature importance.

Чуть более быстрый способ - `HashingVectorizer()`, у него обратного преобразования нет, но как небольшой плюс он масштабирует признаки на выходе.

In [ ]:
# Предобработка по группам признаков:
transformers = [
                    ("encoder", CountVectorizer(max_features=500), text_columns),
                    ("drop", "drop", useless_columns),
               ]

preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough', n_jobs=-1, sparse_threshold=0)

In [ ]:
def optimize(model, params, X, y):
    name = f'{type(model).__name__}'
    print(f'Оптимизация {name}...')
    pipe = Pipeline([
                        ('preprocessor', preprocessor),
                        ('scaler', None),                
                        ('model', model)
                    ])
    gcv = GridSearchCV(pipe, params, cv=4, scoring='neg_log_loss', n_jobs=-1)
    gcv.fit(X, y)
    print(f'Лучшие гиперпараметры: {dict(gcv.best_params_)}')
    print(f'Скор: {(-gcv.best_score_):.2f}')
    return gcv.best_estimator_

In [ ]:
# Список скейлеров:
scaler_list = [
               StandardScaler(),
               PowerTransformer(),
               QuantileTransformer(random_state=RANDOM_STATE),
               QuantileTransformer(random_state=RANDOM_STATE, output_distribution='normal'),
              ]

In [ ]:
encoder_list = [
                    CountVectorizer(max_features=500, stop_words='english'),
                    #CountVectorizer(max_features=500, ngram_range=(1,2)),
                    HashingVectorizer(n_features=500, stop_words='english'),
                    TfidfVectorizer(max_features=500, stop_words='english'),
               ]

In [ ]:
lr_params = {
                    #'preprocessor__encoder':encoder_list,
                    'scaler':scaler_list,
                    'model__C':np.logspace(-3, 3, 7),
              }

In [ ]:
best_lr = optimize(LogisticRegression(solver='newton-cholesky', random_state=RANDOM_STATE, n_jobs=-1), lr_params, X_train, y_train)

In [ ]:
visualize(y_test, best_lr.predict_proba(X_test)[:,1])

In [ ]:
mdi_importances = pd.Series(best_lr['model'].coef_[0], index=best_lr[:-1].get_feature_names_out())

In [ ]:
mdi_importances.sort_values(ascending=False).head(10)

#### N-grams

Можно строить признаки не только на отдельных словах, но и на словах, встречающихся в паре (или в более длинной последовательности). Для этого достаточно воспользоваться параметром `ngram_range`, например вот так:

In [ ]:
CountVectorizer(max_features=200, ngram_range=(2,2))

### TF-IDF

Этот способ учитывает употребление слова как в конкретной записи, так и во всем столбце.

Term Frequency:

$$
TF = \frac{t}{n}
$$

$t$ - число повторов слова в тексте, $n$ - длина текста

Inverse Document Frequency:

$$
IDF = log_{10}\frac{D}{d}
$$

$D$ - общее число текстов, $d$ - число текстов, где слово встречается.

Реализован в `TfidfVectorizer()`.

### Самостоятельная работа

Измените гиперпараметры выше, чтобы обучить модель на биграмах (можно использовать любой векторайзер). Стало ли лучше? Как думаете, почему?

## Предобработка текста

Для классических методов нам желательно не включать ничего лишнего в данные, чтобы получить рабочее и компактное преобразование. В этом могут помочь:

- удаление пунктуации;
- чистка от специфического мусора (гиперссылки, эмодзи и т. п.);
- задание списка стоп-слов, которые встречаются часто, но мало добавляют к смыслу (артиклей, предлогов, местоимений...);
- лемматизация (приведение слова к словарной форме);
- стемминг (выделение основы слова).

Основные библиотеки для исследования и обработки текстов:

- NLTK
- spaCy
- gensim
- TextBlob

На примере библиотеки gensim:

In [ ]:
sentence = X_train['text'].iloc[9]
sentence

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, strip_non_alphanum

In [ ]:
strip_non_alphanum(sentence)

In [ ]:
remove_stopwords(sentence)

In [ ]:
from gensim.parsing.porter import PorterStemmer
p = PorterStemmer()
p.stem_sentence(sentence)

### Токенизация

Наконец, для обработки предложение надо разбить на список слов (векторайзеры из sklearn делают это сами, но им можно указать свои функции для предобработки и токенизации). В большинстве случаев достаточно `split()`, но если надо, например, сохранить пунктуацию, процесс усложняется.

`preprocess_string()` из gensim - один из вариантов пакетной обработки, от чистки до стемминга и токенизации:

In [ ]:
preprocess_string(sentence)

Библиотеки, предоставляющие лемматизацию: https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/

In [ ]:
def clean(text):
    return ' '.join(preprocess_string(text))

In [ ]:
lr_params = {
                    'preprocessor__encoder':[CountVectorizer(max_features=500, preprocessor=clean)],
                    'scaler':scaler_list,
                    'model__C':np.logspace(-3, 3, 7),
              }

In [ ]:
best_lr = optimize(LogisticRegression(solver='newton-cholesky', random_state=RANDOM_STATE, n_jobs=-1), lr_params, X_train, y_train)

In [ ]:
visualize(y_test, best_lr.predict_proba(X_test)[:,1])

## Word2Vec (на примере gensim)

In [ ]:
from gensim.models import Word2Vec

import gensim.downloader as api

Можно обучить Word2Vec на наших данных, но результат будет так себе. Для качественных эмбеддингов требуется большой корпус данных.

In [ ]:
embeddings_trained = Word2Vec(
                                X_train['text'].apply(preprocess_string),
                                vector_size=100,
                                min_count=25,
                                window=25,
                                workers=-1,
                             ).wv

К счастью, предобученных моделей много в свободном доступе:

In [ ]:
model_glove_twitter = api.load("glove-twitter-100")

### Свойства векторов эмбеддингов

In [ ]:
for key, similarity in model_glove_twitter.similar_by_word('cat')[:5]:
    print(f"{key}: {similarity:.2f}")

In [ ]:
new_vector = model_glove_twitter['kitten'] - model_glove_twitter['cat'] +  model_glove_twitter['dog']

In [ ]:
for key, similarity in model_glove_twitter.similar_by_vector(new_vector)[:1]:
    print(f"{key}: {similarity:.2f}")

In [ ]:
model_glove_twitter.most_similar(positive=['kitten', 'dog'], negative=['cat'])

### Совместимость предобработки

In [ ]:
list(model_glove_twitter.key_to_index)[:20]

In [ ]:
import re

In [ ]:
def process_hashtag(hashtag):
    hashtag_body = hashtag[1:]
    if hashtag_body.isupper():
        result = "<HASHTAG> {} <ALLCAPS>".format(hashtag_body)
    else:
        result = "<HASHTAG> {}".format(" ".join(re.findall(r'[A-Z][^A-Z]*', hashtag_body)))

    return result
def preprocess_twitter(input):
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    input = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<URL>", input)
    input = re.sub("/", " / ", input)
    input = re.sub(r"@\w+", "<USER>", input)
    input = re.sub(r"{0}{1}[)d]+|[)d]+{1}{0}".format(eyes, nose), "<SMILE>", input, flags=re.IGNORECASE)
    input = re.sub(r"{0}{1}p+".format(eyes, nose), "<LOLFACE>", input, flags=re.IGNORECASE)
    input = re.sub(r"{0}{1}\(+|\)+{1}{0}".format(eyes, nose), "<SADFACE>", input)
    input = re.sub(r"{0}{1}[\/|l*]".format(eyes, nose), "<NEUTRALFACE>", input)
    input = re.sub("<3", "<HEART>", input)
    input = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<NUMBER>", input)
    input = re.sub(r"#\S+", lambda hashtag: process_hashtag(hashtag.group(0)), input)
    input = re.sub(r"([!?.]){2,}", lambda match: match.group(1) + " <REPEAT>", input)
    input = re.sub(r"\b(\S*?)(.)\2{2,}\b", lambda match: match.group(1) + match.group(2) + " <ELONG>", input)
    #input = re.sub(r"\b[A-Z][A-Z]+\b", lambda word: word.group(0).lower() + " <ALLCAPS>", input)
    #input = re.sub(r"([^a-z0-9()<>'`\-]){2,}", lambda word: word.group(0).lower() + " <ALLCAPS>", input)

    return re.sub(r"[^a-z <>]",' ', input.lower()).split()

In [ ]:
X_train['text'].iloc[0]

In [ ]:
preprocess_twitter(X_train['text'].iloc[9])

### Как построить эмбеддинг для фразы

In [ ]:
def sentence_to_vec(sentence, embeddings=model_glove_twitter, dim=100):
    result = []
    token_list = preprocess_twitter(sentence)
    for token in token_list:
        if token in embeddings:
            result.append(embeddings[token])
    return np.mean(result, axis=0) if result else np.zeros(dim)

### Обучение классификатора на эмбеддингах

In [ ]:
X_train_embed = np.array([sentence_to_vec(x) for x in X_train['text'].values])
X_test_embed = np.array([sentence_to_vec(x) for x in X_test['text'].values])

In [ ]:
lr_params = {
                    'preprocessor':[None],
                    'scaler':scaler_list,
                    'model__C':np.logspace(-3, 3, 7),
            }

In [ ]:
best_lr = optimize(LogisticRegression(solver='newton-cholesky', random_state=RANDOM_STATE, n_jobs=-1), lr_params, X_train_embed, y_train)

In [ ]:
visualize(y_test, best_lr.predict_proba(X_test_embed)[:,1])

## Проекция на меньшую размерность

In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = ['Noto Sans CJK JP', 'sans-serif']

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def plot_w2v(model, w2v=None, save=None):
    if save:
        fig, ax = plt.subplots(figsize=(128,128))
    plt.scatter(model[:, 0], model[:, 1], s=1);
    if save:
        for i, v in enumerate(w2v.vectors[:10000]):
            word = w2v.index_to_key[i]
            plt.annotate(word, (model[i, 0], model[i, 1]))
        plt.savefig(save)

In [ ]:
pca = PCA(n_components=2, random_state=RANDOM_STATE).fit_transform(model_glove_twitter.vectors[:10000])
plot_w2v(pca)

### t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, metric='cosine', n_jobs=-1, random_state=RANDOM_STATE).fit_transform(model_glove_twitter.vectors[:10000])

In [ ]:
plot_w2v(tsne)

### UMAP

In [ ]:
from umap import UMAP

In [ ]:
umap = UMAP(n_components=2, metric='cosine', n_neighbors=5, random_state=RANDOM_STATE).fit_transform(model_glove_twitter.vectors[:10000])
plot_w2v(umap)

## Кластеризация

### KMeans

In [ ]:
from skfuzzy.cluster import cmeans

In [ ]:
def elbow_plot(data, max_clusters=20):
    metrics = []
    
    for i in range(1, max_clusters+1):
        result = cmeans(data, c=i, m=5.0, error=5e-3, maxiter=1000, seed=177013)
        metrics.append(result[4][-1])
    
    plt.plot(range(1, max_clusters+1), metrics)
    plt.title('График локтя')
    plt.xlabel('Число кластеров')
    plt.ylabel('Среднее расстояние до центра')
    plt.xlim((1, max_clusters))
    plt.ylim(0)

In [ ]:
elbow_plot(umap.T)

In [ ]:
from skfuzzy.cluster import cmeans

In [ ]:
result = cmeans(umap.T, c=10, m=2.0, error=5e-3, maxiter=1000, seed=177013)

In [ ]:
preds = np.argmax(result[1], axis=0)

In [ ]:
centers = result[0]

In [ ]:
plt.scatter(umap[:, 0], umap[:, 1], c=preds, cmap='icefire');
plt.scatter(centers[:, 0], centers[:,1], marker='+', s=400, color='red');

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(n_clusters=10, random_state=RANDOM_STATE, n_init='auto')
kmeans = model.fit_predict(umap)
plt.scatter(umap[:, 0], umap[:, 1], c=kmeans, cmap='icefire');
plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[:, 1], marker='+', s=400, color='red');

In [ ]:
def elbow_plot(data, method, max_clusters=20):
    metrics = []
    
    for i in range(1, max_clusters+1):
        model = method(n_clusters=i, random_state=RANDOM_STATE, n_init='auto')
        model.fit(data)
        metrics.append(model.inertia_)
    
    plt.plot(range(1, max_clusters+1), metrics)
    plt.title('График локтя')
    plt.xlabel('Число кластеров')
    plt.ylabel('Среднее расстояние до центра')
    plt.xlim((1, max_clusters))
    plt.ylim(0)

In [ ]:
elbow_plot(umap, KMeans)

In [ ]:
model = KMeans(n_clusters=5, random_state=RANDOM_STATE, n_init='auto')
kmeans = model.fit_predict(umap)
plt.scatter(umap[:, 0], umap[:, 1], c=kmeans, cmap='icefire');
plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[:, 1], marker='+', s=400, color='red');

### Иерархическая кластеризация

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
hier = AgglomerativeClustering(n_clusters=5)
agg_result = hier.fit_predict(umap)
plt.scatter(umap[:, 0], umap[:, 1], c=agg_result, cmap='icefire');

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
#dendrogram(linkage(umap, method='ward'), truncate_mode='level');
plt.title('Дендрограмма')
plt.ylabel('Расстояние');

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=1, min_samples=5, n_jobs=-1)
dbs = dbscan.fit_predict(umap)
plt.scatter(umap[:, 0], umap[:, 1], c=dbs, cmap='icefire');

## Выбросы и аномалии

Мы уже учились выявлять выбросы в рамках отдельных признаков с помощью диаграммы размаха. Здесь мы разберем некоторые продвинутые методы.

In [ ]:
df = pd.read_csv('framingham_heart_disease.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df = df.dropna()

In [ ]:
_, ax = df.groupby('TenYearCHD')['glucose'].hist(bins='fd', figsize=(10,4), alpha=0.5);
ax.set(xlabel='Уровень глюкозы', ylabel='Наблюдений, шт', title='Распределение уровня глюкозы в крови')
ax.legend(['Здоровые', 'Больные']);

In [ ]:
_, ax = df.groupby('TenYearCHD')['BMI'].hist(bins='fd', figsize=(10,4), alpha=0.5);
ax.set(xlabel='ИМТ', ylabel='Наблюдений, шт', title='Распределение ИМТ')
ax.legend(['Здоровые', 'Больные']);

### Применение функций расстояния

In [ ]:
outlier_space = ['BMI', 'glucose']

# Матрица ковариации:
covariance = np.cov(df[outlier_space], rowvar=False)
inv_covariance = np.linalg.inv(covariance)

# Центр:
center = np.mean(df[outlier_space], axis=0)

In [ ]:
from scipy.spatial.distance import mahalanobis

def calc_distance(row):
    return mahalanobis(row, center, inv_covariance)

In [ ]:
distance = df[outlier_space].apply(calc_distance, axis=1)

In [ ]:
# Отсекаем 1% выбросов:
limit = distance.quantile(0.99)

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
lambda_, v = np.linalg.eig(covariance)
lambda_ = np.sqrt(lambda_)
ellipse = Ellipse(xy=(center[0], center[1]),
                  width=lambda_[0] * limit * 2, height=lambda_[1] * limit * 2,
                  angle=np.rad2deg(np.arccos(v[0, 0])))
ellipse.set_alpha(0.5)
fig = plt.figure(figsize=(10,6))
ax = plt.subplot()
plt.scatter(x=df['BMI'], y=df['glucose']);
ax.add_artist(ellipse)
ax.set(xlim=(0), ylim=(0), title='Границы выбросов', xlabel='BMI', ylabel='Glucose');

In [ ]:
# Comment this out to test without outlier exclusion:
df[distance < limit][outlier_space].agg(['min', 'max']).T

In [ ]:
# df['BMI'] = df['BMI'].apply(lambda x: np.clip(x, 15.54, 44.71))
# df['glucose'] = df['glucose'].apply(lambda x: np.clip(x, 40, 193))

### Изоляционный лес

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
isolation_forest = IsolationForest(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE, contamination=0.01) 
outliers = isolation_forest.fit_predict(df)

In [ ]:
df[outliers==-1].head()

### Метод ближайших соседей для поиска аномалий

In [ ]:
from pyod.models.knn import KNN

In [ ]:
outliers = KNN(contamination=0.01, n_jobs=-1).fit_predict(df) 

In [ ]:
df[outliers==1].head()

# Домашнее задание

## Easy

Вернитесь к любому датасету с категориальными признаками, над которым мы работали (или скачайте новый с интернета). Предложите оптимальный метод кодирования признаков. Реализуйте его (достаточно `.fit_transform()`).

In [ ]:
# Ваш код ниже:


## Normal

Изучите эмбеддинги word2vec, обученные на википедии:

In [ ]:
model_wiki = api.load("glove-wiki-gigaword-300")

Вы можете также воспользоваться другой предобученной моделью или найти ее в интернете:

In [ ]:
list(api.info()['models'].keys())

1. Спроецируйте эмбеддинги на плоскость. Возможно, вам придется перебрать несколько методов и гиперпараметров, чтобы получить более показательную картину.

In [ ]:
# Ваш код ниже:


2. Попробуйте выделить кластеры методом DBSCAN. Сделайте вывод по наблюдениям. Насколько вы довольны результатом?

In [ ]:
# Ваш код ниже:


Сохранить визуализацию в файл можно с помощью написанной выше функции `plot_w2v(my_umap, w2v=model_wiki, save='file.png')`.

## Hard

Попробуйте реализовать простую модель Word2Vec методом градиентного спуска. Обучающую выборку возьмите по своему усмотрению.

Вам понадобится:

- сделать one-hot encoding слов (`CountVectorizer` подойдет);
- придумать, как легко определять, в одном ли контексте слова, и обучать на соответствующих парах;
- возможно, собирать вектор контекста для слова;
- обучить матрицу эмбеддингов: если слово встречается в контексте, вектора должны быть похожи, если нет - ортогональны.

Изучите полученные эмбеддинги. Довольны ли вы результатом?